In [1]:
import sys
import os
import numpy as np
import pandas as pd
import random
import matplotlib.pyplot as plt
import statsmodels.api as sm
from sklearn.linear_model import LinearRegression as LinearRegression
# Add the src directory t\ the Python path

sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))
# Now you can import the TimeSeriesDataset class
from src.TimeSeriesDataset import TimeSeriesDataset as TSD
from src.utils import download_data, weighted_mean
from src.mappings import value_mapping, ideology_mapping, social_complexity_mapping

In [2]:
polity_df = download_data("https://seshatdata.com/api/core/polities/")
dataset = TSD(categories=['sc',"wf"], file_path="../datasets/power_transitions.csv")

Downloaded 839 rows
Dataset loaded from ../datasets\power_transitions.xlsx


In [3]:
import pyreadr

data = pyreadr.read_r("../datasets/TableData.Rdata")
df = data['ImpSCDat']
df_agri = data['AggrSCWarAgriRelig']

add_cols = ['Metal','Project','Weapon','Animal','Armor','Defense','Cavalry','Iron','Agri','AgriLag']
for col in add_cols:
    df[col] = df_agri[col]

col_replacements = {'Time': 'Year', 'Hier':'Hierarchy', 'Gov':'Government', 'Infra':'Infrastructure',
                    'Info': 'Information'}
df = df.rename(columns=col_replacements)
df['PolityName'] = df['PolID'].apply(lambda x: polity_df.loc[polity_df['name'] == x, 'new_name'].values[0] if x in polity_df['name'].values else np.nan)
df['PolityID'] = df['PolID'].apply(lambda x: polity_df.loc[polity_df['name'] == x, 'id'].values[0] if x in polity_df['name'].values else np.nan)
df.drop_duplicates(subset=['PolityName', 'Year'], inplace=True)
df_old = df.copy()

In [4]:
add_cols = ['Agri','AgriLag']
for col in add_cols:
    dataset.scv[col] = np.nan
# add cols to dataset by merging on PolityID and year from df_agri
for n, row in df.iterrows():
    for col in add_cols:
        dataset.scv.loc[(dataset.scv['PolityName'] == row['PolityName']) & (dataset.scv['Year'] == row['Year']), col] = row[col]
        if len(dataset.scv.loc[(dataset.scv['PolityName'] == row['PolityName']) & (dataset.scv['Year'] == row['Year'])]) > 1:
            print(row['PolityName'], row['Year'])
            

for pol in dataset.scv.PolityName.unique(): 
    for col in add_cols:
        if dataset.scv.loc[dataset.scv['PolityName'] == pol, col].isna().any() and dataset.scv.loc[dataset.scv['PolityName'] == pol, col].notna().any():
            if col == 'Agri':
                dataset.scv.loc[dataset.scv['PolityName'] == pol, col] = dataset.scv.loc[dataset.scv['PolityName'] == pol, col].fillna(method='ffill')
            elif col == 'AgriLag':
                # calculate change in year and add to agrilag value
                agrilag = dataset.scv.loc[(dataset.scv['PolityName'] == pol)&(dataset.scv["AgriLag"].notna()), col].iloc[0]
                year = dataset.scv.loc[(dataset.scv['PolityName'] == pol)&(dataset.scv["AgriLag"].notna()), 'Year'].iloc[0]
                year_diff = year - dataset.scv.loc[(dataset.scv['PolityName'] == pol), 'Year']
                dataset.scv.loc[dataset.scv['PolityName'] == pol, col] = dataset.scv.loc[dataset.scv['PolityName'] == pol, col].fillna(agrilag - year_diff)

transfer_cols = ["NGA","PolityID","PolityName", "Year", 'Agri','AgriLag','Miltech','IronCav','Cavalry']
for col in transfer_cols:
    dataset.scv_imputed[col] = dataset.scv[col]

it_st_peter_rep_1 900
it_st_peter_rep_1 900
it_papal_state_4 1700
it_papal_state_4 1700
it_venetian_rep_3 1400
it_venetian_rep_3 1400
eg_new_k_1 -1400
eg_new_k_1 -1400
eg_new_k_2 -1200
eg_new_k_2 -1200
eg_mamluk_sultanate_3 1500
eg_mamluk_sultanate_3 1500
af_kushan_emp 100
af_kushan_emp 100
in_gupta_emp 500
in_gupta_emp 500
kh_angkor_1 1000
kh_angkor_1 1000
cn_northern_song_dyn 1100
cn_northern_song_dyn 1100
jp_yayoi 200
jp_yayoi 200
jp_azuchi_momoyama 1600
jp_azuchi_momoyama 1600
us_hawaii_3 1700
us_hawaii_3 1700
es_spanish_emp_1 1700
es_spanish_emp_1 1700


In [5]:
def get_shifted_values(dataset):
    """
    This function calculates the shifted values for Scale and Comp columns in the dataset.
    It creates new columns for the shifted values and the differences between the current and shifted values.
    Finally it saves the new columns in the dataset.
    """
    dr = dataset.scv_imputed.copy()
    # remove rows without NGA value
    dr = dr.dropna(subset=['NGA'])
    dr['Scale_t'] = np.nan
    dr['d_Scale'] = np.nan
    dr['Comp_t'] = np.nan
    dr['d_Comp'] = np.nan


    dr_nga = dr.groupby('NGA')
    for nga, group in dr_nga:
        group = group.sort_values(['Year', 'PolityID'])
        group.drop_duplicates(subset = ['Year'], keep = 'first', inplace = True)

        # find values 100 years later
        dr.loc[group.index, 'Scale_t'] = group['Scale_1'].shift(-1).values
        dr.loc[group.index, 'Comp_t'] = group['Comp'].shift(-1).values
        dr.loc[group.index, 'd_Scale'] = dr.loc[group.index, 'Scale_t'] - group['Scale_1']
        dr.loc[group.index, 'd_Comp'] = dr.loc[group.index, 'Comp_t'] - group['Comp']
        
        # remove rows without a value 100 yrs later
        mask = (group['Year'].shift(-1) - group['Year'] != 100)
        indices = group.index.intersection(mask[mask].index)

        dr.loc[indices,'Scale_t'] = np.nan
        dr.loc[indices,'Comp_t'] = np.nan
        dr.loc[indices,'d_Scale'] = np.nan
        dr.loc[indices,'d_Comp'] = np.nan
    dataset.scv_imputed = dr.copy()


def sample_polities(dataset):
    n_polities = len(dataset.scv_imputed['PolityName'].unique())
    new_dataset = pd.DataFrame(columns=dataset.scv_imputed.columns)
    for i in range(n_polities):
        # sample polities with replacement
        polity = random.choices(dataset.scv_imputed['PolityName'].unique())[0]
        polity_data = dataset.scv_imputed[dataset.scv_imputed['PolityName'] == polity]
        new_dataset = pd.concat([new_dataset, polity_data])
    new_dataset.reset_index(drop=True, inplace=True)
    return new_dataset

In [30]:
N_imputations = 20
N_boot = 50
linear_models = pd.DataFrame(columns = ['n_imputations','n_boot', "fit", 'r2', 'r2_adj', 'aic', 'bic'])
scv = dataset.scv.copy()
scv_imputed = dataset.scv_imputed.copy()
for n_imputations in range(N_imputations):
    # reset the dataset to the original state
    dataset.scv_imputed = scv_imputed[transfer_cols].copy()
    dataset.scv = scv.copy()

    dataset.scv['Hierarchy_sq'] = dataset.scv['Hierarchy']**2
    # impute scale and non scale variables separately
    scale_cols = ['Pop','Terr','Cap','Hierarchy', 'Hierarchy_sq']
    dataset.impute_missing_values(scale_cols, use_duplicates = False, r2_lim=0., add_resid=True)
    non_scale_cols = ['Government', 'Infrastructure', 'Information', 'Money']
    dataset.impute_missing_values(non_scale_cols, use_duplicates = False, r2_lim=0., add_resid=True)

    scale_pca_cols = ['Pop','Terr','Cap']
    dataset.compute_PCA(cols = scale_pca_cols, col_name = 'Scale', n_cols = 1, n_PCA= len(scale_pca_cols), contributions=False, rescale=False)
    lm_df = dataset.scv_imputed[['Pop', 'Scale_1']].dropna()
    X = lm_df[['Pop']]
    y = lm_df['Scale_1']

    # Create and fit the model
    model = LinearRegression()
    model.fit(X, y)

    # Extract the coefficients
    intercept = model.intercept_
    slope = model.coef_[0]
    dataset.scv_imputed['Scale_1'] = (dataset.scv_imputed['Scale_1'] - intercept) / slope
    comp_mapping = {'Comp':{'Government': 11, 'Infrastructure': 12, 'Information':13, 'Money': 6}}
    dataset.scv_imputed['Comp'] = dataset.scv_imputed.apply(lambda row: weighted_mean(row, comp_mapping,category = 'Comp',imputation = "remove", min_vals=0.5), axis=1)

    dataset.scv = dataset.scv.groupby('dataset').get_group('100y')
    dataset.scv_imputed["dataset"] = dataset.scv["dataset"]
    dataset.scv_imputed = dataset.scv_imputed.groupby('dataset').get_group('100y')
    get_shifted_values(dataset)
    
    for n_boot in range(N_boot):
        x_cols = ['Scale_1','Scale_sq','Agri','IronCav', 'Comp']
        y_col = ['Scale_t']
        Xy = sample_polities(dataset)
        Xy['Scale_sq'] = Xy['Scale_1']**2
        Xy = Xy[y_col+x_cols].dropna()

        Xy['intercept'] = 1
        X = Xy[x_cols+ ['intercept']]
        X.drop(columns=['Comp'], inplace=True)
        y = Xy[y_col]

        model = sm.OLS(y, X).fit()
        predictions = model.predict(X) # make the predictions by the model

        model_row = pd.DataFrame([[n_imputations, n_boot, model, model.rsquared, model.rsquared_adj, model.aic, model.bic]], columns = linear_models.columns)
        linear_models = pd.concat([linear_models, model_row], ignore_index=True)

there are some NaNs in the imputed dataset
Explained variance by each PC:
PC1: 81.88%
PC2: 11.12%
PC3: 7.01%


C:\Users\matil\AppData\Local\Temp\ipykernel_6188\287880222.py:49: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X.drop(columns=['Comp'], inplace=True)
C:\Users\matil\AppData\Local\Temp\ipykernel_6188\287880222.py:49: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X.drop(columns=['Comp'], inplace=True)
C:\Users\matil\AppData\Local\Temp\ipykernel_6188\287880222.py:49: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X.drop(columns=['Comp'], 

there are some NaNs in the imputed dataset
Explained variance by each PC:
PC1: 82.24%
PC2: 11.30%
PC3: 6.46%


C:\Users\matil\AppData\Local\Temp\ipykernel_6188\287880222.py:49: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X.drop(columns=['Comp'], inplace=True)
C:\Users\matil\AppData\Local\Temp\ipykernel_6188\287880222.py:49: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X.drop(columns=['Comp'], inplace=True)
C:\Users\matil\AppData\Local\Temp\ipykernel_6188\287880222.py:49: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X.drop(columns=['Comp'], 

there are some NaNs in the imputed dataset
Explained variance by each PC:
PC1: 82.32%
PC2: 10.77%
PC3: 6.91%


...

C:\Users\matil\AppData\Local\Temp\ipykernel_6188\287880222.py:49: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X.drop(columns=['Comp'], inplace=True)
C:\Users\matil\AppData\Local\Temp\ipykernel_6188\287880222.py:49: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X.drop(columns=['Comp'], inplace=True)
C:\Users\matil\AppData\Local\Temp\ipykernel_6188\287880222.py:49: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X.drop(columns=['Comp'], 

there are some NaNs in the imputed dataset
Explained variance by each PC:
PC1: 82.33%
PC2: 11.00%
PC3: 6.68%


C:\Users\matil\AppData\Local\Temp\ipykernel_6188\287880222.py:49: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X.drop(columns=['Comp'], inplace=True)
C:\Users\matil\AppData\Local\Temp\ipykernel_6188\287880222.py:49: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X.drop(columns=['Comp'], inplace=True)
C:\Users\matil\AppData\Local\Temp\ipykernel_6188\287880222.py:49: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X.drop(columns=['Comp'], 

there are some NaNs in the imputed dataset
Explained variance by each PC:
PC1: 81.90%
PC2: 11.34%
PC3: 6.76%


C:\Users\matil\AppData\Local\Temp\ipykernel_6188\287880222.py:49: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X.drop(columns=['Comp'], inplace=True)
C:\Users\matil\AppData\Local\Temp\ipykernel_6188\287880222.py:49: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X.drop(columns=['Comp'], inplace=True)
C:\Users\matil\AppData\Local\Temp\ipykernel_6188\287880222.py:49: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X.drop(columns=['Comp'], 

In [ ]:
parameters = linear_models["fit"].apply(lambda x: x.params)

for col in parameters.columns:
    print(f"column: {col}")
    print(f"mean: {parameters[col].mean()}")
    print(f"standard error: {parameters[col].std()/np.sqrt(len(parameters))}")
    # calculate t-statistic and p-value
    t_stat = parameters[col].mean() / (parameters[col].std()/np.sqrt(len(parameters)))
    from scipy.stats import t
    p_value = 2 * (1 - t.cdf(np.abs(t_stat), df=len(parameters)-1))
    print(f"t-statistic: {t_stat}")
    print(f"p-value: {p_value}\n")

column: Scale_1
mean: 0.5719340858202586
standard error: 0.0037531777903403963
t-statistic: 152.386622155831
p-value: 0.0

column: Scale_sq
mean: 0.02680303949070123
standard error: 0.0003605850108103808
t-statistic: 74.33209558673536
p-value: 0.0

column: Agri
mean: 0.217865912191207
standard error: 0.0016913954651261115
t-statistic: 128.80838141242313
p-value: 0.0

column: IronCav
mean: 0.1433347307298561
standard error: 0.001111320951791588
t-statistic: 128.97689951654618
p-value: 0.0

column: intercept
mean: 1.1609356616237558
standard error: 0.008726297239960577
t-statistic: 133.038748245642
p-value: 0.0



In [ ]:
# def compute_bootstraps_imputation(self, cols, n_nga_bootstraps = 100, n_resid_boostraps = 20, use_duplicates = False, r2_lim = 0.0):

    #     # create empty dataframes
    #     all_imputed = pd.DataFrame(columns = self.scv.columns)
    #     original_scv = self.scv.copy()
    #     original_scv_imputed = self.scv_imputed.copy()
    #     if len(self.scv_imputed) == 0:
    #         original_scv_imputed = self.scv.copy()
    #     self.scv.loc[self.scv.NGA.isna(), 'NGA'] = "Unknown"
    #     ngas = self.scv.NGA.unique()

    #     for n in range(n_nga_bootstraps):
    #         # create a new dataframe with the same columns as the original scv
    #         new_scv = pd.DataFrame(columns = self.scv.columns)
            
    #         while len(new_scv) < len(self.scv):
    #             # sample a NGA from the original scv
    #             nga = np.random.choice(ngas, 1)[0]
    #             # add rows from the original scv to the new scv
    #             nga_rows = original_scv.loc[self.scv.NGA == nga].copy()
    #             if len(nga_rows) == 0:
    #                 continue
    #             new_scv = pd.concat([new_scv, nga_rows], ignore_index=True)
            
    #         for i in range(n_nga_bootstraps):
    #             # compute the imputation for the new scv
    #             self.scv = new_scv.copy()
    #             self.imputation_fits = pd.DataFrame([])
    #             self.scv_imputed = pd.DataFrame([])
    #             self.impute_missing_values(cols, use_duplicates=use_duplicates, r2_lim=r2_lim, add_resid=True)
    #             # add the imputed values to the all_imputed dataframe
    #             all_imputed = pd.concat([all_imputed, self.scv_imputed], ignore_index=True)

    #     self.scv_imputed = original_scv_imputed
    #     errors = pd.DataFrame(0, columns=cols, index=original_scv.index)
    #     # reconstruct imputed csv
    #     for index, row in original_scv.iterrows():
    #         # find positions of nans
    #         year = row['Year']
    #         polity = row['PolityID']
    #         row = row[cols]
    #         nan_cols = row[row.isna()].index
    #         non_nan_cols = row[row.notna()].index
    #         if len(non_nan_cols) == 0:
    #             continue
    #         for col in nan_cols:
    #             # find the rows in all_imputed where the columns are not nan
    #             imputed_vals = all_imputed.loc[(all_imputed['Year'] == year) & (all_imputed['PolityID'] == polity), col]
    #             imputed_vals = imputed_vals.dropna()
    #             # check if imputed_vals is empty    
    #             if len(imputed_vals) == 0:
    #                 continue
    #             # check if imputed_vals is a series
    #             else:
    #                 imputed_val = imputed_vals.mean()
    #                 self.scv_imputed.loc[index, col] = imputed_val
    #                 # calculate the error
    #                 errors.loc[index, col] = imputed_vals.std()
        
    #     self.scv = original_scv